In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

In [22]:
import pandas as pd

In [23]:
import time

In [24]:
dates = [3,4,5,6]
url = 'https://apps-congress.eular.org/eular2020/en-GB/PublicProgram/SessionTableJson?'
Masterdf = pd.DataFrame()
for i in tqdm_notebook(dates):
    pDay = '2020-06-0' + str(i) + 'T06:00:00'
    params = {'pDay': pDay,'pEventId':'7','pProgramGrade':'0','pShowPrivate':'false'}
    data = requests.get(url,params = params)
    dict_ = data.json()
    for j in tqdm_notebook(dict_['Sessions']):
        id = j['Id']
        sessionUrl = 'https://apps-congress.eular.org/eular2020/en-GB/ProgramSearch/Program/?searchstring=&programid='+str(id)+'&pProgramGrade=All&pHideLogin=False&pHidePersonal=False'
        time.sleep(7)
        sessionData = requests.get(sessionUrl)
        try:
            dfList = pd.read_html(sessionData.text)
            df = dfList[-1].copy()
            df['Room'] = j['RoomName']
            df['Session Name'] = j['Name']
            df['Session Type'] = j['SessionTypeName']
            df['Session Start Time'] = j['StartTimeString']
            df['Session End Time'] = j['EndTimeString']
            df['Chairs'] = j['Chairs']
            
            df['Description'] = j['Description']
            df.drop(columns = ['Unnamed: 0','Unnamed: 4','Unnamed: 5','Appointment'],axis=1,inplace = True)
            Masterdf = Masterdf.append(df,ignore_index = True)
        except:
            print('Yes')
#           df = pd.DataFrame()
            dict_1 = {}    
            dict_1['Room'] = j['RoomName']
            dict_1['Session Type'] = j['SessionTypeName']
            dict_1['Session Start Time'] = j['StartTimeString']
            dict_1['Session End Time'] = j['EndTimeString']
            dict_1['Session Name'] = j['Name']
            dict_1['Chairs'] = j['Chairs']
            dict_1['Description'] = j['Description']
#             print(df)
            Masterdf = Masterdf.append(dict_1,ignore_index = True)
        

C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


Yes



ConnectionError: HTTPSConnectionPool(host='apps-congress.eular.org', port=443): Max retries exceeded with url: /eular2020/en-GB/ProgramSearch/Program/?searchstring=&programid=585&pProgramGrade=All&pHideLogin=False&pHidePersonal=False (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000021B7166DC08>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [8]:
Masterdf2 = Masterdf.copy()

Code for Poster data

In [13]:
from selenium import webdriver
import time

In [14]:
BackSchedule = '//*[@id="btnSchedule"]'

In [15]:
xpath = '//*[@id="Days"]/a'

In [16]:


DayDate = '//*[@id="Days"]/button[2]'

In [18]:
dates = [2,3,4]
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')
LinkPosters = []
driver.get('https://apps-congress.eular.org/eular2020/en-GB/PublicProgram')
for i in dates:
    DayDate = '//*[@id="Days"]/button[' + str(i) + ']'
    driver.find_element_by_xpath(DayDate).click()
    time.sleep(1)
    driver.find_element_by_xpath(xpath).click()
    time.sleep(1)
    data = driver.page_source
    data = BeautifulSoup(data,'lxml')
    # pd.read_html(data)
    posterContainer = data.find('div',id = 'postersContainer')
    lnks = posterContainer.find_all('div',class_ = 'poster ng-scope')
    for j in lnks:
        LinkPosters.append('https://apps-congress.eular.org/' + j.find_all('a')[0]['href'])
    driver.find_element_by_xpath(BackSchedule).click()


In [19]:
len(LinkPosters)

37

In [150]:
count = 1
for i in tqdm_notebook(LinkPosters):
    
    data = requests.get(i)
    data = BeautifulSoup(data.text,'lxml')
    heading = data.find('h4').text
    location = str(data.select('#content > div.container-fluid > div > div')[0]).split('<br/>\n')[0].split('</i>')[-1].strip()
    SessionType = str(data.select('#content > div.container-fluid > div > div')[0]).split('<br/>\n')[2].split('</i>')[-1].strip()
    SessionTime = str(data.select('#content > div.container-fluid > div > div')[0]).split('<br/>\n')[3].split('</i>')[-1].strip('<span id="dateString">').strip('</span><br/')
    try:
        dfList = pd.read_html(i)
        df = dfList[-1].copy()
        df['Room'] = location
        df['Session Name'] = heading
        df['Session Type'] = SessionType
        df['Poster Time'] = SessionTime
        #             df['Session End Time'] = j['EndTimeString']
        #             df['Chairs'] = j['Chairs']

        #             df['Description'] = j['Description']
        df.drop(columns = ['Unnamed: 0','Unnamed: 4','Unnamed: 5','Appointment'],axis=1,inplace = True)
        Masterdf = Masterdf.append(df,ignore_index = True)
    except:
        print('Yes',count)
        count+=1
        #           df = pd.DataFrame()
        dict_1 = {}    
        dict_1['Room'] = location
        dict_1['Session Type'] = SessionType
        dict_1['Poster Time'] = SessionTime
        #             dict_1['Session End Time'] = j['EndTimeString']
        dict_1['Session Name'] = heading
        #             dict_1['Chairs'] = j['Chairs']
        #             dict_1['Description'] = j['Description']
        #             print(df)
        Masterdf = Masterdf.append(dict_1,ignore_index = True)


In [151]:
Masterdf3 = Masterdf.copy()

In [152]:
Masterdf['Date'] = Masterdf['Time'].str.split('2020',expand = True)[0]

In [153]:
Masterdf['Start Time'] = Masterdf['Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[0]
Masterdf['End Time'] = Masterdf['Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[1]

In [154]:
Masterdf['Poster Date'] = Masterdf['Poster Time'].str.split('2020',expand = True)[0]

In [155]:
Masterdf['Poster Start Time'] = Masterdf['Poster Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[0]
Masterdf['Poster End Time'] = Masterdf['Poster Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[1]

In [157]:
Masterdf

,Chairs,Description,Poster Time,Room,Session End Time,Session Name,Session Start Time,Session Type,Time,Title,Date,Start Time,End Time,Poster Date,Poster Start Time,Poster End Time
0,None,<ul><li>To be able to discuss the key activiti...,NaN,Festhalle,14:00,Opening plenary session,13:00,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,<ul><li>To understand the increased fracture r...,NaN,Festhalle,14:55,WIN - Bone fragility in chronic kidney disease,14:15,WIN session,03 June 2020 14:20 - 14:55,What is new - Bone disease and fracture risk i...,03 June,14:20,14:55,NaN,NaN,NaN
2,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,NaN,Hall 3A,15:45,tsDMARD therapy of rheumatoid arthritis,14:15,Clinical Science session,03 June 2020 14:20 - 14:45,JAKi in the news – side effects MD Roy Fleisc...,03 June,14:20,14:45,NaN,NaN,NaN
3,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,NaN,Hall 3A,15:45,tsDMARD therapy of rheumatoid arthritis,14:15,Clinical Science session,03 June 2020 14:45 - 15:10,“Selective” JAKi on the verve – implications f...,03 June,14:45,15:10,NaN,NaN,NaN
4,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,NaN,Hall 3A,15:45,tsDMARD therapy of rheumatoid arthritis,14:15,Clinical Science session,03 June 2020 15:10 - 15:35,Ten years of JAKi in RA – Real life data from ...,03 June,15:10,15:35,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,NaN,NaN,06 June 2020 10:30 - 12:10,Poster area (Exhibition hall),NaN,\r\n Rehabilitation\r\n ...,NaN,Poster tour HPR,06 June 2020 11:20 - 11:30,SAT0611-HPR Low adherence to recommended physi...,06 June,11:20,11:30,06 June,10:30,12:10
1040,NaN,NaN,06 June 2020 10:30 - 12:10,Poster area (Exhibition hall),NaN,\r\n Rehabilitation\r\n ...,NaN,Poster tour HPR,06 June 2020 11:30 - 11:40,SAT0612 Team-rehabilitation benefits body comp...,06 June,11:30,11:40,06 June,10:30,12:10
1041,NaN,NaN,06 June 2020 10:30 - 12:10,Poster area (Exhibition hall),NaN,\r\n Rehabilitation\r\n ...,NaN,Poster tour HPR,06 June 2020 11:40 - 11:50,SAT0613-HPR Effect of Cervical Stabilization E...,06 June,11:40,11:50,06 June,10:30,12:10
1042,NaN,NaN,06 June 2020 10:30 - 12:10,Poster area (Exhibition hall),NaN,\r\n Rehabilitation\r\n ...,NaN,Poster tour HPR,06 June 2020 11:50 - 12:00,SAT0614-HPR Identifying and optimising multipl...,06 June,11:50,12:00,06 June,10:30,12:10


In [163]:
len(Masterdf.columns)


16

In [164]:
cols = ['Session Name','Room','Session Type','Session Start Time','Session End Time','Description', 'Chairs', 'Time','Date', 'Title', 
       'Start Time', 'End Time','Poster Time','Poster Date', 'Poster Start Time',
       'Poster End Time']
len(cols)
Masterdf = Masterdf[cols]

In [170]:
Masterdf['Session Name'] = Masterdf['Session Name'].str.strip()

In [174]:
Masterdf['Session Start Time']

0       13:00
1       14:15
2       14:15
3       14:15
4       14:15
        ...  
1039      NaN
1040      NaN
1041      NaN
1042      NaN
1043      NaN
Name: Session Start Time, Length: 1044, dtype: object

In [175]:
Masterdf['Session Start Time'] = Masterdf['Session Start Time'].fillna(Masterdf['Poster Start Time'])

In [176]:

Masterdf['Session End Time'] = Masterdf['Session End Time'].fillna(Masterdf['Poster End Time'])

In [165]:
Masterdf['Description'] = Masterdf2['Description'].str.strip().str.replace('&lt;ul&gt;&lt;li&gt;','').str.replace('&lt;/li&gt;&lt;/ul&gt;','').str.replace('ul type="disc"','').str.replace('&lt','').str.replace('/li','').str.replace('&gt','').str.replace(';','').str.replace('<ul>','').str.replace('<li>','').str.replace('/','').str.replace('<p>','').str.replace('<','').str.replace('span lang="EN-US"','').str.replace('span','').str.replace('>','').str.replace('nbsp','').str.replace('&','').str.replace('middot','').str.replace('lang="EN-GB"','')
Masterdf['Chairs'] = Masterdf['Chairs'].str.replace('<br>',',')

In [166]:
Masterdf['Description'][686]

'Take home messages for HPR after the EULAR Frankfurt 2020 Congress.'

In [44]:
Masterdf['Description']

0      To be able to discuss the key activities of EU...
1      To understand the increased fracture risk in p...
2      To get an up-to-date information on oral thera...
3      To get an up-to-date information on oral thera...
4      To get an up-to-date information on oral thera...
                             ...                        
686    Take home messages for HPR after the EULAR Fra...
687    Take home messages for HPR after the EULAR Fra...
688    Take home messages for HPR after the EULAR Fra...
689    To provide an overview of learning and outcome...
690    To provide an overview of learning and outcome...
Name: Description, Length: 691, dtype: object

In [177]:
Masterdf.to_excel('Eular2020.xls')